#### install package

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import RobustScaler, StandardScaler
import datetime 

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format

## 지역별 (모델 학습)

### 날짜 지역 가격 INDEX 쇼핑몰 코드별로 CSV파일 생성

In [ ]:
final_df = pd.read_pickle('./fc주문_통합_수정.pkl')

In [ ]:
final_df.head()

,CORP_ID,REF_ORD_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_CD,ITEM_QTY,ITEM_AMT,REF_ITEM_SEQ,DLVPREARRBRANCD,DLVPREARREMPNICKNM,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2,개별가격,평균가격,metric,cluster
0,KX007,2021030365922551,7,2021-03-03,035510,90001302,22400.00000,134265392,1,14900.00000,2021030323885041.0,7484,C06,5Z73,1,3.0,세종특별자치시,nan,서울,서대문구,14900.00000,14900.00000,134265392,8.0
1,KX007,2021030365922951,7,2021-03-03,035521,90001441,20900.00000,4610261043,1,20900.00000,2021030323885721.0,6427,M09,6R67,4,50.0,경기도,광주시,경남,김해시,20900.00000,20529.68342,4610261043,0.0
2,KX007,2021030365924361,7,2021-03-03,035513,90001441,37900.00000,4936599538,1,37900.00000,2021030323887981.0,6158,L32,9Y25,0,53.0,경기도,광주시,경남,거제시,37900.00000,27175.94846,4936599538,8.0
3,KX007,2021030365928921,7,2021-03-03,035608,90001542,69800.00000,3858391637,1,69800.00000,2021030323895281.0,8909,D21,1N65,1,6.0,경기도,광주시,서울,서초구,69800.00000,102245.81445,3858391637,8.0
4,KX007,2021030365930451,7,2021-03-03,035551,90001341,73700.00000,4631307460,1,39800.00000,2021030323897651.0,2777,R15,3D84,1,10.0,경기도,광주시,경기,김포시,39800.00000,39800.00000,4631307460,8.0


In [ ]:
#클러스터 빈거 10으로 채움
final_df['cluster'].replace({'nan':'10'},inplace = True)

In [ ]:
#평균가격 구하기(단순 나누기), 날짜별로 달라지는걸 보여주고 싶음
final_df['ITEM_PRICE'] = final_df.apply(lambda x: x['ITEM_AMT']/x['ITEM_QTY'], axis = 1)

In [ ]:
final_df.SHPR_CD.unique()

array(['90001302', '90001441', '90001542', '90001341', '90001541',
       '90001443', '90001381', '90001521', '90001582', '90001602',
       '90001662', '90001622', '90001682', '90001683', '90001702',
       '90001703', '90001705', '90001704', '90001664', '90001768',
       '90001765', '90001776', '90001774', '90001842'], dtype=object)

In [ ]:
final_df.columns

Index(['CORP_ID', 'REF_ORD_NO', 'BKG_TYP', 'BKG_DATE', 'BKG_TIME', 'SHPR_CD',
       'INV_AMT', 'ITEM_CD', 'ITEM_QTY', 'ITEM_AMT', 'REF_ITEM_SEQ',
       'DLVPREARRBRANCD', 'DLVPREARREMPNICKNM', 'DLVCLSFCD', 'DLVSUBCLSFCD',
       'POST_ZONE', 'SHPR_ADDR_1', 'SHPR_ADDR_2', 'CNEE_ADDR_1', 'CNEE_ADDR_2',
       '개별가격', '평균가격', 'metric', 'cluster', 'ITEM_PRICE'],
      dtype='object')

In [ ]:
demo = pd.read_pickle("./demographic.pkl")

In [ ]:
demo.head()

,시도,month,10대 비율,"2,30대 비율","4,5,60대 비율",남성 비율,여성 비율,총인구수,1인,2인,3인,4인,5인,전체가구수
0,강원,03,0.08860,0.22280,0.47770,0.50330,0.49670,1536399.00000,0.43360,0.25750,0.15220,0.11540,0.04130,735644.00000
1,강원,04,0.08840,0.22230,0.47850,0.50320,0.49680,1536175.00000,0.43440,0.25780,0.15190,0.11470,0.04120,735644.00000
2,강원,05,0.08820,0.22200,0.47920,0.50340,0.49660,1535491.00000,0.43440,0.25850,0.15180,0.11440,0.04090,736980.00000
3,강원,06,0.08810,0.22170,0.47980,0.50340,0.49660,1535530.00000,0.43500,0.25900,0.15150,0.11390,0.04060,738068.00000
4,경기,03,0.09840,0.27580,0.45410,0.50330,0.49670,13465837.00000,0.35980,0.22390,0.18700,0.17670,0.05260,5719333.00000


In [ ]:
demo.columns

Index(['시도', 'month', '10대 비율', '2,30대 비율', '4,5,60대 비율', '남성 비율', '여성 비율',
       '총인구수', '1인', '2인', '3인', '4인', '5인', '전체가구수'],
      dtype='object')

In [ ]:
demo.columns = ['CNEE_ADDR_1', 'month', '10대 비율', '2,30대 비율', '4,5,60대 비율', '남성 비율', '여성 비율',
       '총인구수', '1인', '2인', '3인', '4인', '5인', '전체가구수']

In [ ]:
hol = pd.read_csv('./holiday_cleaned.csv', encoding = 'utf-8')

In [ ]:
hol.head()

,날짜,설명,요일,휴일여부
0,2018-01-01,새해,0,1
1,2018-01-02,해당없음,1,0
2,2018-01-03,해당없음,2,0
3,2018-01-04,해당없음,3,0
4,2018-01-05,해당없음,4,0


In [ ]:
hol['날짜'] = hol['날짜'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))

In [ ]:
hol = hol[(hol['날짜']>='2021-03-01') &(hol['날짜']<='2021-06-30')]

In [ ]:
hol.head()

,날짜,설명,요일,휴일여부
1155,2021-03-01,삼일절,0,1
1156,2021-03-02,해당없음,1,0
1157,2021-03-03,해당없음,2,0
1158,2021-03-04,해당없음,3,0
1159,2021-03-05,해당없음,4,0


In [ ]:
hol = hol[['날짜','요일','휴일여부']]

In [ ]:
hol.columns = ['BKG_DATE', '요일','휴일여부']

In [ ]:
final_df.columns

Index(['CORP_ID', 'REF_ORD_NO', 'BKG_TYP', 'BKG_DATE', 'BKG_TIME', 'SHPR_CD',
       'INV_AMT', 'ITEM_CD', 'ITEM_QTY', 'ITEM_AMT', 'REF_ITEM_SEQ',
       'DLVPREARRBRANCD', 'DLVPREARREMPNICKNM', 'DLVCLSFCD', 'DLVSUBCLSFCD',
       'POST_ZONE', 'SHPR_ADDR_1', 'SHPR_ADDR_2', 'CNEE_ADDR_1', 'CNEE_ADDR_2',
       '개별가격', '평균가격', 'metric', 'cluster', 'ITEM_PRICE'],
      dtype='object')

In [ ]:
clist = final_df.CORP_ID.unique()

In [ ]:
ilist = final_df.CNEE_ADDR_1.unique()

### 누락 컬럼 생성

In [ ]:
# shpr_cd 분리 때문에 비어있는 컬럼 확인하기 위해 컬럼 생성
cllt = list(final_df.cluster.unique())
tlt = cllt

In [ ]:
# 누락 되어있는 컬럼으로 dataframe 생성
def make_df(temp):
  collist = []
  for i in tlt:
    if i in temp.columns:
      pass
    else : 
      collist.append(i)
  cdf = pd.DataFrame(columns = collist)
  return cdf

### 최종 전처리

In [ ]:
def preprocessing(df, demo, hol):
  date = pd.DataFrame(pd.period_range('2021-03-01', '2021-06-30'))
  date.columns = ['BKG_DATE']
  date['BKG_DATE'] = date['BKG_DATE'].apply(lambda x: x.to_timestamp())
  for c in clist :
    for i in ilist:
      i = str(i)
      # temp_df : df를 cnee_addr별로 끊어 쓰기
      temp_df = df[(df['CNEE_ADDR_1']==i)&(df['CORP_ID']==c)]
      demo_df = demo[demo['CNEE_ADDR_1']==i]

      # 일별로 1개의 컬럼만 존재하게끔 output dataframe 구성
      tmp =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()
      tmp1 =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()
      tmp2 =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()
      tmp3 =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()

      # 일별 group by한 dataframe을 0301-0630사이 날짜 dataframe에 merge
      # 매출이 없는 날의 경우, 결측값을 0으로 처리
      tmp = pd.merge(date, tmp, on = 'BKG_DATE', how='left')
      tmp.fillna(0, inplace = True)
      tmp['month'] = tmp['BKG_DATE'].apply(lambda x: str(x)[5:7])
  
      # 휴일, 세대 가구수, 성별 연령별 merge
      tmp = pd.merge(tmp, demo_df, how = 'left', on = 'month')
      tmp = pd.merge(tmp, hol, how = 'left', on = 'BKG_DATE')

      # 하루전, 2일전, 3일전 판매개수
      tmp1['BKG_DATE']=tmp1['BKG_DATE']+datetime.timedelta(1)
      tmp2['BKG_DATE']=tmp2['BKG_DATE']+datetime.timedelta(2)
      tmp3['BKG_DATE']=tmp3['BKG_DATE']+datetime.timedelta(3)
      tmp1=tmp1.rename(columns={'ITEM_QTY':'DAY_1'})
      tmp2=tmp2.rename(columns={'ITEM_QTY':'DAY_2'})
      tmp3=tmp3.rename(columns={'ITEM_QTY':'DAY_3'})
      DAY_df = pd.merge(tmp, tmp1, how = 'left', on=['BKG_DATE'])
      DAY_df = pd.merge(DAY_df, tmp2, how = 'left', on=['BKG_DATE'])
      DAY_df = pd.merge(DAY_df, tmp3, how = 'left', on=['BKG_DATE'])
      
      # 첫날 개수는 0으로 처리
      DAY_df=DAY_df.fillna(0)
      # 주별 물품 개수 합계
      DAY_df['week'] = DAY_df['BKG_DATE'].apply(lambda x: x.isocalendar()[1])
      DAY_df['week'] = DAY_df['week'].apply(lambda x: x-8)
      dfn= DAY_df.groupby(['week'])['ITEM_QTY'].sum().reset_index()
      dfn=dfn.rename(columns={'ITEM_QTY':'WEEK_AMT'})
      last_df = pd.merge(DAY_df, dfn, how = 'left', on=['week'])

      # 일별 평균 가격
      df3=temp_df.groupby(['BKG_DATE'])['ITEM_PRICE'].mean().reset_index()
      df3=df3.rename(columns={'ITEM_PRICE':'MEAN_PRICE'})
      last_df = pd.merge(last_df, df3, how = 'left', on = ['BKG_DATE'])

      # 클러스터별 판매량 group by 후 pivot table
      cneedf = temp_df.groupby(['BKG_DATE','cluster'])['ITEM_QTY'].sum().reset_index()
      cneedf = cneedf.pivot(index = 'BKG_DATE', columns = 'cluster', values = 'ITEM_QTY')
      last_df = pd.merge(last_df, cneedf, how = 'left', on = 'BKG_DATE')
      last_df.fillna(0, inplace = True)

      cdf = make_df(last_df)
      final = pd.concat([last_df, cdf], axis=1)
      final.fillna(0, inplace=True)
      final = final[['BKG_DATE', 'ITEM_QTY', '10대 비율', '2,30대 비율',
       '4,5,60대 비율', '남성 비율', '여성 비율', '총인구수', '1인', '2인', '3인', '4인', '5인',
       '전체가구수', '요일', '휴일여부', 'DAY_1', 'DAY_2', 'DAY_3', 'WEEK_AMT',
       'MEAN_PRICE', '0.0', '1.0', '10', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '8.0', '9.0']]

      final.to_pickle("./corp_id_"+c+"_cnee_addr_"+i+".pkl")



In [ ]:
preprocessing(final_df, demo, hol)